### Fine-Tune Command

In [1]:
!pip install transformers[torch] datasets peft bitsandbytes tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 373.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 268.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 904.9 kB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip

In [1]:
import json

curated_ds = json.load(open("data_5000.json"))

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/pythia-160m"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["### End", "### Instruction:", "### Response:\n", "### Context:"]}
)

model = AutoModelForCausalLM.from_pretrained(model_id, revision="step130000")
# model = AutoModelForCausalLM.from_pretrained(model_id, revision="step143000")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
curated_ds["data"][0]

{'command': '"Maintain regular exercise, eat balanced meals, and get enough sleep."',
 'search_term': '"Staying Healthy Tips"',
 'explanation': 'To maintain optimal physical and mental well-being, it is essential to incorporate simple yet effective habits into your daily routine, such as drinking plenty of water, exercising regularly, getting enough sleep, and maintaining a balanced diet.',
 'instruction_prompt': '### Instruction: "Maintain regular exercise, eat balanced meals, and get enough sleep." \n\n### Context: To maintain optimal physical and mental well-being, it is essential to incorporate simple yet effective habits into your daily routine, such as drinking plenty of water, exercising regularly, getting enough sleep, and maintaining a balanced diet. \n\n### Response:\n',
 'original_instruction': 'Give three tips for staying healthy.',
 'original_input': '',
 'original_output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regul

In [4]:
def tokenize_alpaca_item(tokenizer, item):
    text = f'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n{item["instruction_prompt"]}{item["original_output"]}\n\n### End### End### End'
    return tokenizer(text)

In [5]:
import tqdm

ds_tokenized = [tokenize_alpaca_item(tokenizer, item) for item in tqdm.tqdm(curated_ds["data"])]


100%|██████████| 5000/5000 [00:02<00:00, 2056.23it/s]


In [6]:
for tmp in tqdm.tqdm(ds_tokenized):
    # print(tmp['input_ids'][-1])
    # print(tokenizer("### End"))
    assert tmp['input_ids'][-1]==tokenizer("### End")['input_ids'][0]

100%|██████████| 5000/5000 [00:00<00:00, 23163.28it/s]


In [7]:
from peft import LoraConfig, get_peft_model, PeftModel

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM",
) 

peft_model = get_peft_model(model, config)
print_trainable_parameters(peft_model)

trainable params: 294912 || all params: 162617856 || trainable%: 0.18135277838123753


In [8]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=ds_tokenized,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        # max_steps=100,
        num_train_epochs=1,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=5,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/gpt_neox/modeling_gpt_neox.py", line 1203, in forward
    lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 152

In [9]:
item = curated_ds["data"][0]
print(item)

PROMPT = f'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: Write a sentence telling me about: {item["search_term"]}\n\n### Response:\n'
answer = tokenizer.decode(peft_model.to('cuda').generate(**tokenizer(PROMPT, return_tensors="pt").to('cuda'), max_new_tokens=50, temperature=0.99, top_k=500, do_sample=True)[0])
print(answer.split("### End")[0].split("### Response:\n")[1])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'command': '"Maintain regular exercise, eat balanced meals, and get enough sleep."', 'search_term': '"Staying Healthy Tips"', 'explanation': 'To maintain optimal physical and mental well-being, it is essential to incorporate simple yet effective habits into your daily routine, such as drinking plenty of water, exercising regularly, getting enough sleep, and maintaining a balanced diet.', 'instruction_prompt': '### Instruction: "Maintain regular exercise, eat balanced meals, and get enough sleep." \n\n### Context: To maintain optimal physical and mental well-being, it is essential to incorporate simple yet effective habits into your daily routine, such as drinking plenty of water, exercising regularly, getting enough sleep, and maintaining a balanced diet. \n\n### Response:\n', 'original_instruction': 'Give three tips for staying healthy.', 'original_input': '', 'original_output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly t

In [16]:
item = curated_ds["data"][1001]
print(item)
print(item["search_term"])
PROMPT = f'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: Write a sentence telling me about: {item["search_term"]}\n\n### Response:\n'

answer = tokenizer.decode(peft_model.to('cuda').generate(**tokenizer(PROMPT, return_tensors="pt").to('cuda'), max_new_tokens=50)[0])
print(answer.split("### End")[0].split("### Response:\n")[1].split("\n")[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'command': 'Examine how individual and societal actions impact outcomes.', 'search_term': '"Effects of Individual vs Societal Performance"', 'explanation': 'The comparison between individual and societal performance in terms of achievement and productivity reveals that while individuals may excel in specific domains, societal achievements are often driven by the collective efforts and interdependence of group members.', 'instruction_prompt': '### Instruction: Examine how individual and societal actions impact outcomes. \n\n### Context: The comparison between individual and societal performance in terms of achievement and productivity reveals that while individuals may excel in specific domains, societal achievements are often driven by the collective efforts and interdependence of group members. \n\n### Response:\n', 'original_instruction': 'Compare and contrast the effects of individual and societal performance', 'original_input': '', 'original_output': 'Individual performance is rel

In [17]:
peft_model.save_pretrained("v1_pythia_160m_step130000-explanation")